# [0] pip install

In [ ]:
# !pip install azure-identity
# !pip install azure-storage-blob
# !pip install azure-storage-file-datalake
# !pip install azure.storage.blob

In [ ]:

import os, uuid, sys
from azure.identity import DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings


# [1] loop through directories

In [ ]:
# adls2source
storage_account_name = "<storage-account-name>"
storage_account_key = "<storage-account-key>"

# Authorize access and connect to data resources  authorized with the account key
def initialize_storage_account(storage_account_name, storage_account_key):
    try:  
        global service_client       
        service_client = DataLakeServiceClient(
            account_url="{}://{}.dfs.core.windows.net".format("https", storage_account_name), 
            credential=storage_account_key
            )
    except Exception as e:
        print(e)


In [ ]:
initialize_storage_account(
    storage_account_name=storage_account_name, 
    storage_account_key=storage_account_key
)

In [ ]:
def list_directory_contents():
    try:
        file_system_client = service_client.get_file_system_client(file_system="<container-name>")
        paths = file_system_client.get_paths(path="<sub-dir-name>")
        for path in paths:
            print(path.name)
    except Exception as e:
     print(e)


In [ ]:
list_directory_contents()

# [2] walk through ALL Sub-Directory && Files in Storage Container 

In [ ]:
import os
import sys
import json
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import BlobPrefix

In [ ]:
def walk_container(client, container_name):

    container_client = client.get_container_client(container_name)

    # Replace "<connection_string>" with your actual connection string
    connection_string = "DefaultEndpointsProtocol=https;AccountName=<storage-account-name>;AccountKey=<storage-account-key>;EndpointSuffix=core.windows.net"

    # Create a BlobServiceClient object
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Specify the container name
    container_name = "<container-name>"

    # print(container_name)
    for _sub1 in container_client.walk_blobs():
        # print(_sub1.name)
        for _sub2 in container_client.list_blobs(_sub1.name):
            if _sub2.content_settings.content_type == "text/csv":
                print(f"{_sub2.container}/{_sub2.name}")
                # print(_sub2.content_settings.content_type)
            else:
                pass


In [ ]:

# Call the walk_container function
walk_container(blob_service_client, container_name)


# [3] create python class to get all files in container

In [ ]:
class GetFilesList():
    
    def __init__(self, container_name, client):
        # self.container_name = container_name
        self.connection_string = "DefaultEndpointsProtocol=https;AccountName=adls2source;AccountKey=<storage-account-key>;EndpointSuffix=core.windows.net"

        # # Create a BlobServiceClient object
        self.blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        self.client = self.blob_service_client

        # # Specify the container name
        self.container_name = "<container-name>"

    def walk_container(self):

        container_client = self.client.get_container_client(self.container_name)
        # print(self.container_name)
        for _sub1 in container_client.walk_blobs():
            # print(_sub1.name)
            for _sub2 in container_client.list_blobs(_sub1.name):
                if _sub2.content_settings.content_type == "text/csv":
                    print(f"{_sub2.container}/{_sub2.name}")
                    # print(_sub2.content_settings.content_type)
                else:
                    pass


In [ ]:
_list = GetFilesList("<container-name>", blob_service_client)
_list.walk_container()
